In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date, timedelta
import re
import sys
import os
import xlrd
import io

In [13]:
# ======= SETTINGS =========
# etp_toggle = 'ETF'
etp_toggle = 'LIP'

### set target date; can set to today
target_date = datetime(2018, 11, 28)

if etp_toggle == 'ETF':
    aum_col_name = "AUM (Fund Total) ('000)"
else:
    aum_col_name = "AUM (L&I Product Total) ('000)"
    

In [14]:
### assume at most 15 holidays
timedelta_max = 35
past_date_ls = []

timedelta_range = range(1, timedelta_max + 1)

### retrieve data at 12pm each day
# target_date = datetime.today()
# dt = datetime.now()
# if dt.hour < 12:
#     timedelta_range = range(-1, timedelta_max - 1)
# else:
#     timedelta_range = range(0, timedelta_max)
    
for i in timedelta_range:    
    past_date = target_date - timedelta(timedelta_max - i)
    past_date_str = past_date.strftime('%Y%m%d')
    past_date_ls.append(past_date_str)
print(past_date_str)

20181128


In [7]:
csv_names = [str(past_date + '-' + etp_toggle + 'List.csv') for past_date in past_date_ls]
xlsx_names = [str(past_date + '-' + etp_toggle + 'List.xlsx') for past_date in past_date_ls]
print('First day files: ', csv_names[0], xlsx_names[0])

First day files:  20181025-ETFList.csv 20181025-ETFList.xlsx


In [15]:
stock_code_ls = pd.read_excel('code_list_hk_all_' + etp_toggle + 's_v20181024.xlsx', header=None)
stock_code_ls = stock_code_ls[0].tolist()

In [16]:
# check the latest number
data = pd.read_excel('/home/lena/jupyter-notebook/Lena-RA-ETF/HKEX/xlsx-version/%s_' % past_date_str + etp_toggle + 'List.xls', skiprows=1) 
data["T/O"] = data["T/O ('000)"] * 1000
# data["T/O"]

In [17]:
# finalized-T/O: in specified currency
T_O_table = pd.DataFrame(columns=stock_code_ls)

for past_date_str in past_date_ls:
    data = pd.read_excel('/home/lena/jupyter-notebook/Lena-RA-ETF/HKEX/xlsx-version/%s_' % past_date_str + etp_toggle + 'List.xls', skiprows=1) 
    
    data_date = pd.read_excel('/home/lena/jupyter-notebook/Lena-RA-ETF/HKEX/xlsx-version/%s_' % past_date_str + etp_toggle + 'List.xls') 
    data_date = data_date.columns[0]
    data_date = data_date[data_date.find(": ") + 2:]
    data_date_dtobj = datetime.strptime(data_date, '%d/%m/%Y')
    
    data["T/O"] = data["T/O ('000)"] * 1000

    df_tmp = pd.DataFrame([data["T/O"].tolist()], index=[data_date_dtobj.strftime('%Y-%m-%d')], columns=stock_code_ls)
    T_O_table = T_O_table.append(df_tmp)
    
T_O_table.drop_duplicates(inplace=True)

if len(T_O_table) >= 22:
    T_O_table = T_O_table.iloc[len(T_O_table)-22:, :]    
else:
    print('insufficient days: at least 22 days of data is needed, only %d days found' % len(T_O_table))
# T_O_table.head()

filename_T_O_raw = '/home/lena/jupyter-notebook/Lena-RA-ETF/hkex_' + etp_toggle + '_22d_T_O_in_dollars_raw_v%s.xlsx' % past_date_ls[-1]
filename_T_O = '/home/lena/jupyter-notebook/Lena-RA-ETF/hkex_' + etp_toggle + '_22d_avg_T_O_in_dollars_v%s.xlsx' % past_date_ls[-1]

writer = pd.ExcelWriter(filename_T_O_raw)
T_O_table.to_excel(writer, sheet_name = 'hk_' + etp_toggle + '_22dTO_dollarraw_' + past_date_ls[-1])
writer.save()
print("File saved: %s" % filename_T_O_raw)

T_O_table_1 = T_O_table.copy()

writer = pd.ExcelWriter(filename_T_O)
T_O_table_1.mean().to_frame().to_excel(writer, sheet_name = 'hk-' + etp_toggle + '-avg-TO-dollar-' + past_date_ls[-1])
writer.save()
print("File saved: %s" % filename_T_O)

File saved: /home/lena/jupyter-notebook/Lena-RA-ETF/hkex_LIP_22d_T_O_in_dollars_raw_v20181128.xlsx
File saved: /home/lena/jupyter-notebook/Lena-RA-ETF/hkex_LIP_22d_avg_T_O_in_dollars_v20181128.xlsx


In [18]:
# finalized
# closing price: in specified currency
closing_price_table = pd.DataFrame(columns=stock_code_ls)

past_date_str = past_date_ls[-1]

data = pd.read_excel('/home/lena/jupyter-notebook/Lena-RA-ETF/HKEX/xlsx-version/%s_' % past_date_str + etp_toggle + 'List.xls', skiprows=1) 

data_date = pd.read_excel('/home/lena/jupyter-notebook/Lena-RA-ETF/HKEX/xlsx-version/%s_' % past_date_str + etp_toggle + 'List.xls') 
data_date = data_date.columns[0]
data_date = data_date[data_date.find(": ") + 2:]
data_date_dtobj = datetime.strptime(data_date, '%d/%m/%Y')

# df_tmp = pd.DataFrame([ls_tmp], index=['20180919'])
df_tmp = pd.DataFrame([data["Closing Price"].tolist()], index=["Closing Price"], columns=stock_code_ls)
closing_price_table = closing_price_table.append(df_tmp)
closing_price_table = closing_price_table.transpose()
# closing_price_table.head(), len(closing_price_table)

outstanding_shares_df = pd.read_excel('hkex-' + etp_toggle + '-oustanding-shares-v20181128.xlsx', index=False)
outstanding_shares_df.set_index('index_code_hk', inplace=True)
# outstanding_shares_df.head(), len(outstanding_shares_df)

if len(closing_price_table) == len(outstanding_shares_df):
    print('len(closing_price_table) = len(outstanding_shares_df)')
else:
    print('len(closing_price_table) != len(outstanding_shares_df)')
    
# AUM table: in specified currency, for checking with calculated Market Cap
aum_table = pd.DataFrame(columns=stock_code_ls)

# past_date_str = past_date_ls[-1]

# data = pd.read_excel('/home/lena/jupyter-notebook/Lena-RA-ETF/HKEX/xlsx-version/%s_' % past_date_str + etp_toggle + 'List.xls', skiprows=1) 

# data_date = pd.read_excel('/home/lena/jupyter-notebook/Lena-RA-ETF/HKEX/xlsx-version/%s_' % past_date_str + etp_toggle + 'List.xls') 
# data_date = data_date.columns[0]
# data_date = data_date[data_date.find(": ") + 2:]
# data_date_dtobj = datetime.strptime(data_date, '%d/%m/%Y')

# df_tmp = pd.DataFrame([ls_tmp], index=['20180919'])
df_tmp = pd.DataFrame([data[aum_col_name].tolist()], index=[aum_col_name], columns=stock_code_ls)
aum_table = aum_table.append(df_tmp)
aum_table = aum_table.transpose()
# aum_table

mc_df = closing_price_table.merge(outstanding_shares_df, left_index=True, right_index=True)
mc_df["market_cap ('000)"] = mc_df["Closing Price"] * mc_df.outstanding_shares / 1000
mc_df = mc_df.merge(aum_table, left_index=True, right_index=True)

writer = pd.ExcelWriter('/home/lena/jupyter-notebook/Lena-RA-ETF/hkex_' + etp_toggle + '_market_cap_v%s.xlsx' % past_date_ls[-1])
mc_df.to_excel(writer, sheet_name = 'hk-' + etp_toggle + '-marketcap-' + past_date_ls[-1])
writer.save()

len(closing_price_table) = len(outstanding_shares_df)
